In [116]:
import regex as re
import pandas as pd

In [117]:
f=open('./hadoop_10k.log')
lines=f.readlines()

In [118]:
split_pt=re.compile(r'([\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:,\.](?=\s)|(?<=\b[a-zA-Z]+):(?=\d+|[a-zA-Z]+\b))')
hm_pt=re.compile(r'(.*?\w(?=.\[))(.*)')

In [119]:
header=[]
msg=[]
for line in lines:
    h,m=hm_pt.match(line).groups()
    
    header.append(h)
    msg.append(m)

In [120]:
print(header[0])
print(msg[0])

2015-10-19 14:24:56,892 INFO
 [IPC Server handler 13 on 39673] org.apache.hadoop.mapred.TaskAttemptListenerImpl: Progress of TaskAttempt attempt_1445182159119_0004_m_000000_0 is : 0.54488474


In [121]:
header_tokens=[]
for h in header:
    tokens=[token for token in split_pt.split(h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [122]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [123]:
pd.DataFrame({'header_tokens':header_tokens,'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [124]:
header_anno=[]
for tokens in header_tokens:
    anno=['O']*len(tokens)
    anno[0],anno[2],anno[4]='B-DATE','B-TIME','B-LVL'

    header_anno.append(anno)

In [125]:
list(zip(header_tokens[0],header_anno[0]))

[('2015-10-19', 'B-DATE'),
 (' ', 'O'),
 ('14:24:56,892', 'B-TIME'),
 (' ', 'O'),
 ('INFO', 'B-LVL')]

In [126]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['<O>']*len(tokens)
    msg_anno.append(anno)

In [127]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/'
re_ip=r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [128]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in patt_tag_list:
        _sub_tag(ntokens,ntags,patt,tag)

In [129]:
sub_tag(msg_tokens,msg_anno,
    [(re_cls,'B-CLS'),
    (re_func,'B-FUNC'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_host,'B-HOST'),
    (re_url,'B-URL')])

In [130]:
list(zip(msg_tokens[0],msg_anno[0]))

[(' ', '<O>'),
 ('[', '<O>'),
 ('IPC', '<O>'),
 (' ', '<O>'),
 ('Server', '<O>'),
 (' ', '<O>'),
 ('handler', '<O>'),
 (' ', '<O>'),
 ('13', '<O>'),
 (' ', '<O>'),
 ('on', '<O>'),
 (' ', '<O>'),
 ('39673', '<O>'),
 (']', '<O>'),
 (' ', '<O>'),
 ('org.apache.hadoop.mapred.TaskAttemptListenerImpl', 'B-CLS'),
 (':', '<O>'),
 (' ', '<O>'),
 ('Progress', '<O>'),
 (' ', '<O>'),
 ('of', '<O>'),
 (' ', '<O>'),
 ('TaskAttempt', 'B-CLS'),
 (' ', '<O>'),
 ('attempt_1445182159119_0004_m_000000_0', '<O>'),
 (' ', '<O>'),
 ('is', '<O>'),
 (' ', '<O>'),
 (':', '<O>'),
 (' ', '<O>'),
 ('0.54488474', '<O>')]

In [131]:
with open('./header_check.txt','w') as f:
    for tokens,tags in zip(header_tokens,header_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [132]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [133]:
df=pd.DataFrame({
    'tokens':[h+m for h,m in zip(header_tokens,msg_tokens)],
    'tags':[h+m for h,m in zip(header_anno,msg_anno)],
    'raw_log':[h+m for h,m in zip(header,msg)]
    })
df.to_csv('./hadoop_ds.csv')